## 6.1 Flavors of BERT

In [1]:
from transformers import pipeline

In [2]:
nlp = pipeline("fill-mask", model='bert-base-cased')  # BERT-base

print(type(nlp.model))

# Use the reserved in mask_token to mask a word
preds = nlp(f"If you don’t {nlp.tokenizer.mask_token} at the sign, you will get a ticket")

print('If you don’t *** at the sign, you will get a ticket')

for p in preds:
    print(f"Token:{p['token_str']}. Score: {100*p['score']:,.2f}%")

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<class 'transformers.models.bert.modeling_bert.BertForMaskedLM'>
If you don’t *** at the sign, you will get a ticket
Token:look. Score: 51.35%
Token:stop. Score: 39.66%
Token:glance. Score: 1.02%
Token:wait. Score: 0.60%
Token:turn. Score: 0.57%


In [3]:
nlp = pipeline("fill-mask", model='roberta-base')  # RoBERTa

print(type(nlp.model))

preds = nlp(f"If you don’t {nlp.tokenizer.mask_token} at the sign, you will get a ticket")

print('If you don’t *** at the sign, you will get a ticket')

for p in preds:
    print(f"Token:{p['token_str']}. Score: {100*p['score']:,.2f}%")

<class 'transformers.models.roberta.modeling_roberta.RobertaForMaskedLM'>
If you don’t *** at the sign, you will get a ticket
Token: look. Score: 47.69%
Token: stop. Score: 36.82%
Token: stand. Score: 2.54%
Token: stay. Score: 2.52%
Token: wave. Score: 1.01%


In [4]:
nlp = pipeline("fill-mask", model='distilroberta-base')  # Distil-roBERTa

print(type(nlp.model))

preds = nlp(f"If you don’t {nlp.tokenizer.mask_token} at the sign, you will get a ticket")

print('If you don’t *** at the sign, you will get a ticket')

for p in preds:
    print(f"Token:{p['token_str']}. Score: {100*p['score']:,.2f}%")

<class 'transformers.models.roberta.modeling_roberta.RobertaForMaskedLM'>
If you don’t *** at the sign, you will get a ticket
Token: stop. Score: 42.11%
Token: look. Score: 7.53%
Token: park. Score: 4.92%
Token: arrive. Score: 4.65%
Token: sign. Score: 4.27%


In [5]:
nlp = pipeline("fill-mask", model='distilbert-base-cased')  # Using Distil-BERT

print(type(nlp.model))  

preds = nlp(f"If you don’t {nlp.tokenizer.mask_token} at the sign, you will get a ticket")

print('If you don’t *** at the sign, you will get a ticket')

for p in preds:
    print(f"Token:{p['token_str']}. Score: {100*p['score']:,.2f}%")

<class 'transformers.models.distilbert.modeling_distilbert.DistilBertForMaskedLM'>
If you don’t *** at the sign, you will get a ticket
Token:look. Score: 57.47%
Token:stop. Score: 7.37%
Token:glance. Score: 3.74%
Token:arrive. Score: 2.16%
Token:appear. Score: 1.87%


## 6.2 BERT for sequence classification


In [38]:
from transformers import Trainer, TrainingArguments, DistilBertForSequenceClassification, DistilBertTokenizerFast, \
     DataCollatorWithPadding, pipeline
from datasets import load_metric, Dataset
import numpy as np

In [39]:
snips_file = open('../data/snips.train.txt', 'rb')

snips_rows = snips_file.readlines()

snips_rows[:20]

[b'listen O\r\n',
 b'to O\r\n',
 b'westbam B-artist\r\n',
 b'alumb O\r\n',
 b'allergic B-album\r\n',
 b'on O\r\n',
 b'google B-service\r\n',
 b'music I-service\r\n',
 b'PlayMusic\r\n',
 b'\r\n',
 b'add O\r\n',
 b'step B-entity_name\r\n',
 b'to I-entity_name\r\n',
 b'me I-entity_name\r\n',
 b'to O\r\n',
 b'the O\r\n',
 b'50 B-playlist\r\n',
 b'cl\xc3\xa1sicos I-playlist\r\n',
 b'playlist O\r\n',
 b'AddToPlaylist\r\n']

In [40]:
# This code segment parses the snips dataset into a more manageable format

utterances = []
tokenized_utterances = []
labels_for_tokens = []
sequence_labels = []

utterance, tokenized_utterance, label_for_utterances = '', [], []
for snip_row in snips_rows:
    if len(snip_row) == 2:  # skip over rows with no data
        continue
    if ' ' not in snip_row.decode():  # we've hit a sequence label
        sequence_labels.append(snip_row.decode().strip())
        utterances.append(utterance.strip())
        tokenized_utterances.append(tokenized_utterance)
        labels_for_tokens.append(label_for_utterances)
        utterance = ''
        tokenized_utterance = []
        label_for_utterances = []
        continue
    token, token_label = snip_row.decode().split(' ')
    token_label = token_label.strip()
    utterance += f'{token} '
    tokenized_utterance.append(token)
    label_for_utterances.append(token_label)
    

In [41]:
len(labels_for_tokens), len(tokenized_utterances), len(utterances), len(sequence_labels)

(13084, 13084, 13084, 13084)

In [42]:
print(tokenized_utterances[0])
print(labels_for_tokens[0])
print(utterances[0])
print(sequence_labels[0])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
['O', 'O', 'B-artist', 'O', 'B-album', 'O', 'B-service', 'I-service']
listen to westbam alumb allergic on google music
PlayMusic


In [43]:
unique_sequence_labels = list(set(sequence_labels))
unique_sequence_labels

['RateBook',
 'PlayMusic',
 'SearchCreativeWork',
 'SearchScreeningEvent',
 'AddToPlaylist',
 'GetWeather',
 'BookRestaurant']

In [44]:
sequence_labels = [unique_sequence_labels.index(l) for l in sequence_labels]

print(f'There are {len(unique_sequence_labels)} unique sequence labels')

There are 7 unique sequence labels


In [45]:
from functools import reduce

unique_token_labels = list(set(reduce(lambda x, y: x + y, labels_for_tokens)))
labels_for_tokens = [[unique_token_labels.index(_) for _ in l] for l in labels_for_tokens]

print(f'There are {len(unique_token_labels)} unique token labels')

There are 72 unique token labels


In [46]:
print(tokenized_utterances[0])
print(labels_for_tokens[0])
print([unique_token_labels[l] for l in labels_for_tokens[0]])
print(utterances[0])
print(sequence_labels[0])
print(unique_sequence_labels[sequence_labels[0]])

['listen', 'to', 'westbam', 'alumb', 'allergic', 'on', 'google', 'music']
[27, 27, 12, 27, 37, 27, 35, 59]
['O', 'O', 'B-artist', 'O', 'B-album', 'O', 'B-service', 'I-service']
listen to westbam alumb allergic on google music
1
PlayMusic


In [47]:
snips_dataset = Dataset.from_dict(  # hold data for both sequence and token classification
    dict(
        utterance=utterances, 
        label=sequence_labels,
        tokens=tokenized_utterances,
        token_labels=labels_for_tokens
    )
)

snips_dataset = snips_dataset.train_test_split(test_size=0.2)

In [48]:
snips_dataset['train'][0]

{'utterance': 'rate homicide: a year on the killing streets five stars',
 'label': 0,
 'tokens': ['rate',
  'homicide:',
  'a',
  'year',
  'on',
  'the',
  'killing',
  'streets',
  'five',
  'stars'],
 'token_labels': [27, 20, 11, 11, 11, 11, 11, 11, 26, 61]}

In [49]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /Users/sinanozdemir/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at /Users/sinanozdemir/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at /Users/sinanozdemir/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7f

In [50]:
# simple function to batch tokenize utterances with truncation
def preprocess_function(examples):
    return tokenizer(examples["utterance"], truncation=True)

In [51]:
seq_clf_tokenized_snips = snips_dataset.map(preprocess_function, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [52]:
seq_clf_tokenized_snips['train'][0]

{'utterance': 'rate homicide: a year on the killing streets five stars',
 'label': 0,
 'tokens': ['rate',
  'homicide:',
  'a',
  'year',
  'on',
  'the',
  'killing',
  'streets',
  'five',
  'stars'],
 'token_labels': [27, 20, 11, 11, 11, 11, 11, 11, 26, 61],
 'input_ids': [101,
  3446,
  18268,
  1024,
  1037,
  2095,
  2006,
  1996,
  4288,
  4534,
  2274,
  3340,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [53]:
# DataCollatorWithPadding creates batch of data. It also dynamically pads text to the 
#  length of the longest element in the batch, making them all the same length. 
#  It's possible to pad your text in the tokenizer function with padding=True, dynamic padding is more efficient.

data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

In [54]:
# Data Collator will pad data so that all examples are the same input length.
#  Attention mask is how we ignore attention scores for padding tokens

In [55]:
sequence_clf_model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased', 
    num_labels=len(unique_sequence_labels),
)

# set an index -> label dictionary
sequence_clf_model.config.id2label = {i: l for i, l in enumerate(unique_sequence_labels)}

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/sinanozdemir/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinus

In [56]:
sequence_clf_model.config.id2label[0]

'RateBook'

In [57]:
metric = load_metric("accuracy")

def compute_metrics(eval_pred):  # custom method to take in logits and calculate accuracy of the eval set
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)


In [58]:
epochs = 2

training_args = TrainingArguments(
    output_dir="./snips_clf/results",
    num_train_epochs=epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    
    # some deep learning parameters that the Trainer is able to take in
    warmup_steps=len(seq_clf_tokenized_snips['train']) // 5,  # number of warmup steps for learning rate scheduler,
    weight_decay = 0.05,
    
    logging_steps=1,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

# Define the trainer:

trainer = Trainer(
    model=sequence_clf_model,
    args=training_args,
    train_dataset=seq_clf_tokenized_snips['train'],
    eval_dataset=seq_clf_tokenized_snips['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [59]:
# Get initial metrics
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32


{'eval_loss': 1.954656958580017,
 'eval_accuracy': 0.1150171952617501,
 'eval_runtime': 38.6334,
 'eval_samples_per_second': 67.739,
 'eval_steps_per_second': 2.123}

In [60]:
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens.
***** Running training *****
  Num examples = 10467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656


Epoch,Training Loss,Validation Loss,Accuracy
1,0.226400,0.219276,0.977455
2,0.117800,0.061029,0.985097


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
Saving model checkpoint to ./snips_clf/results/checkpoint-328
Configuration saved in ./snips_clf/results/checkpoint-328/config.json
Model weights saved in ./snips_clf/results/checkpoint-328/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
Saving model checkpoint to ./snips_clf/results/checkpoint-656
Configuration saved in ./snips_clf/results/checkpoint-656/config.json
Model weights saved in ./snips_clf/results/checkpoint-656/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface

TrainOutput(global_step=656, training_loss=0.7240334545239443, metrics={'train_runtime': 1182.4158, 'train_samples_per_second': 17.704, 'train_steps_per_second': 0.555, 'total_flos': 117371923027212.0, 'train_loss': 0.7240334545239443, 'epoch': 2.0})

In [61]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32


{'eval_loss': 0.06102892756462097,
 'eval_accuracy': 0.9850974398165838,
 'eval_runtime': 32.8708,
 'eval_samples_per_second': 79.615,
 'eval_steps_per_second': 2.495,
 'epoch': 2.0}

In [110]:
pipe = pipeline("text-classification", sequence_clf_model, tokenizer=tokenizer)
pipe('Add Two Coins by Dispatch to my road trip playlist')

[{'label': 'AddToPlaylist', 'score': 0.9902211427688599}]

In [63]:
trainer.save_model()

Saving model checkpoint to ./snips_clf/results
Configuration saved in ./snips_clf/results/config.json
Model weights saved in ./snips_clf/results/pytorch_model.bin


In [111]:
pipe = pipeline("text-classification", "./snips_clf/results", tokenizer=tokenizer)
pipe('Add Two Coins by Dispatch to my road trip playlist')

loading configuration file ./snips_clf/results/config.json
Model config DistilBertConfig {
  "_name_or_path": "./snips_clf/results",
  "activation": "gelu",
  "architectures": [
    "DistilBertForSequenceClassification"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "RateBook",
    "1": "PlayMusic",
    "2": "SearchCreativeWork",
    "3": "SearchScreeningEvent",
    "4": "AddToPlaylist",
    "5": "GetWeather",
    "6": "BookRestaurant"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "problem_type": "single_label_classification",
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "torch_dtype": "float32",
  "transfor

[{'label': 'AddToPlaylist', 'score': 0.9902211427688599}]

In [65]:
frozen_sequence_clf_model = DistilBertForSequenceClassification.from_pretrained(
    'distilbert-base-uncased',
    num_labels=len(unique_sequence_labels),
)

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/sinanozdemir/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6"
  },
  "initializer_range": 0.02,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5,
    "LABEL_6": 6
  },
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinus

In [115]:
frozen_sequence_clf_model

DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0): TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)
       

In [66]:
for param in frozen_sequence_clf_model.distilbert.parameters():
    param.requires_grad = False  # freezing the parameter so it cannot be updated

In [67]:
epochs = 2

training_args = TrainingArguments(
    output_dir="./snips_clf/results",
    num_train_epochs=epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
    
    warmup_steps=len(seq_clf_tokenized_snips['train']) // 5,
    weight_decay = 0.05,
    
    logging_steps=1,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

# Define the trainer:

trainer = Trainer(
    model=frozen_sequence_clf_model,
    args=training_args,
    train_dataset=seq_clf_tokenized_snips['train'],
    eval_dataset=seq_clf_tokenized_snips['test'],
    compute_metrics=compute_metrics,
    data_collator=data_collator
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [68]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32


{'eval_loss': 1.9464715719223022,
 'eval_accuracy': 0.14902560183416125,
 'eval_runtime': 42.8634,
 'eval_samples_per_second': 61.054,
 'eval_steps_per_second': 1.913}

In [69]:
trainer.train()  # ~20min -> ~6min on my laptop with all of distilbert frozen with a worse loss/accuracy

The following columns in the training set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens.
***** Running training *****
  Num examples = 10467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656


Epoch,Training Loss,Validation Loss,Accuracy
1,1.834200,1.871476,0.433703
2,1.815400,1.583989,0.864348


The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
Saving model checkpoint to ./snips_clf/results/checkpoint-328
Configuration saved in ./snips_clf/results/checkpoint-328/config.json
Model weights saved in ./snips_clf/results/checkpoint-328/pytorch_model.bin
The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
Saving model checkpoint to ./snips_clf/results/checkpoint-656
Configuration saved in ./snips_clf/results/checkpoint-656/config.json
Model weights saved in ./snips_clf/results/checkpoint-656/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface

TrainOutput(global_step=656, training_loss=1.848530449336622, metrics={'train_runtime': 398.3692, 'train_samples_per_second': 52.549, 'train_steps_per_second': 1.647, 'total_flos': 117371923027212.0, 'train_loss': 1.848530449336622, 'epoch': 2.0})

In [70]:
trainer.evaluate()

The following columns in the evaluation set  don't have a corresponding argument in `DistilBertForSequenceClassification.forward` and have been ignored: token_labels, utterance, tokens.
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32


{'eval_loss': 1.5839890241622925,
 'eval_accuracy': 0.8643484906381352,
 'eval_runtime': 31.8316,
 'eval_samples_per_second': 82.214,
 'eval_steps_per_second': 2.576,
 'epoch': 2.0}

## 6.3 BERT for token classification

In [71]:
from transformers import DataCollatorForTokenClassification, DistilBertForTokenClassification, \
                         DistilBertTokenizerFast, pipeline

In [72]:
tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased')

loading file https://huggingface.co/distilbert-base-uncased/resolve/main/vocab.txt from cache at /Users/sinanozdemir/.cache/huggingface/transformers/0e1bbfda7f63a99bb52e3915dcf10c3c92122b827d92eb2d34ce94ee79ba486c.d789d64ebfe299b0e416afc4a169632f903f693095b4629a7ea271d5a0cf2c99
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer.json from cache at /Users/sinanozdemir/.cache/huggingface/transformers/75abb59d7a06f4f640158a9bfcde005264e59e8d566781ab1415b139d2e4c603.7f2721073f19841be16f41b0a70b600ca6b880c8f3df6f3535cbc704371bdfa4
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/distilbert-base-uncased/resolve/main/tokenizer_config.json from cache at /Users/sinanozdemir/.cache/huggingface/transformers/8c8624b8ac8aa99c60c912161f8332de003484428c47906d7f

In [73]:
# Our dataset from the sequence classification section
snips_dataset['train'][0]

{'utterance': 'rate homicide: a year on the killing streets five stars',
 'label': 0,
 'tokens': ['rate',
  'homicide:',
  'a',
  'year',
  'on',
  'the',
  'killing',
  'streets',
  'five',
  'stars'],
 'token_labels': [27, 20, 11, 11, 11, 11, 11, 11, 26, 61]}

In [132]:
tokenized_inputs = tokenizer(snips_dataset['train'][0]["tokens"], truncation=True, is_split_into_words=True)
tokenized_inputs

{'input_ids': [101, 3446, 18268, 1024, 1037, 2095, 2006, 1996, 4288, 4534, 2274, 3340, 102], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [134]:
tokenizer.convert_ids_to_tokens([101, 3446, 18268, 1024, 1037, 2095, 2006, 1996, 4288, 4534, 2274, 3340, 102])

['[CLS]',
 'rate',
 'homicide',
 ':',
 'a',
 'year',
 'on',
 'the',
 'killing',
 'streets',
 'five',
 'stars',
 '[SEP]']

In [133]:
tokenized_inputs.word_ids(batch_index=0)

[None, 0, 1, 1, 2, 3, 4, 5, 6, 7, 8, 9, None]

In [74]:
# The given "token_labels" may not match up with the BERT wordpiece tokenization so
#  this function will map them to the tokenization that BERT uses
#  -100 is a reserved for labels where we do not want to calculate losses so BERT doesn't waste time
#  trying to predict tokens like CLS or SEP

def tokenize_and_align_labels(examples):
    tokenized_inputs = tokenizer(examples["tokens"], truncation=True, is_split_into_words=True)

    labels = []
    for i, label in enumerate(examples[f"token_labels"]):
        word_ids = tokenized_inputs.word_ids(batch_index=i)  # Map tokens to their respective word.
        previous_word_idx = None
        label_ids = []
        for word_idx in word_ids:
            if word_idx is None:  # Set any special tokens / punctuation to -100
                label_ids.append(-100)
            elif word_idx != previous_word_idx:  # Only label the first token of a given word.
                label_ids.append(label[word_idx])
            else:
                label_ids.append(-100)  # everything else labeled as -100
            previous_word_idx = word_idx
        labels.append(label_ids)

    tokenized_inputs["labels"] = labels
    return tokenized_inputs

In [75]:
snips_dataset['train'][0]

{'utterance': 'rate homicide: a year on the killing streets five stars',
 'label': 0,
 'tokens': ['rate',
  'homicide:',
  'a',
  'year',
  'on',
  'the',
  'killing',
  'streets',
  'five',
  'stars'],
 'token_labels': [27, 20, 11, 11, 11, 11, 11, 11, 26, 61]}

In [76]:
# map our dataset from sequence classification to be for token classification
tok_clf_tokenized_snips = snips_dataset.map(tokenize_and_align_labels, batched=True)

  0%|          | 0/11 [00:00<?, ?ba/s]

  0%|          | 0/3 [00:00<?, ?ba/s]

In [128]:
tokenizer.decode([1024])

':'

In [77]:
tok_clf_tokenized_snips['train'][0]

{'utterance': 'rate homicide: a year on the killing streets five stars',
 'label': 0,
 'tokens': ['rate',
  'homicide:',
  'a',
  'year',
  'on',
  'the',
  'killing',
  'streets',
  'five',
  'stars'],
 'token_labels': [27, 20, 11, 11, 11, 11, 11, 11, 26, 61],
 'input_ids': [101,
  3446,
  18268,
  1024,
  1037,
  2095,
  2006,
  1996,
  4288,
  4534,
  2274,
  3340,
  102],
 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
 'labels': [-100, 27, 20, -100, 11, 11, 11, 11, 11, 11, 26, 61, -100]}

In [78]:
tok_clf_tokenized_snips['train'] = tok_clf_tokenized_snips['train'].remove_columns(
    ['utterance', 'label', 'tokens', 'token_labels']
)

tok_clf_tokenized_snips['test'] = tok_clf_tokenized_snips['test'].remove_columns(
    ['utterance', 'label', 'tokens', 'token_labels']
)

tok_clf_tokenized_snips

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 10467
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 2617
    })
})

In [79]:
tok_data_collator = DataCollatorForTokenClassification(tokenizer=tokenizer)

In [80]:
tok_clf_model = DistilBertForTokenClassification.from_pretrained(
    'distilbert-base-uncased', num_labels=len(unique_token_labels)
)

# Set our label dictionary
tok_clf_model.config.id2label = {i: l for i, l in enumerate(unique_token_labels)}

loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at /Users/sinanozdemir/.cache/huggingface/transformers/23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5",
    "6": "LABEL_6",
    "7": "LABEL_7",
    "8": "LABEL_8",
    "9": "LABEL_9",
    "10": "LABEL_10",
    "11": "LABEL_11",
    "12": "LABEL_12",
    "13": "LABEL_13",
    "14": "LABEL_14",
    "15": "LABEL_15",
    "16": "LABEL_16",
    "17": "LABEL_17",
    "18": "LABEL_18",
    "19": "LABEL_19",
    "20": "LABEL_20",
    "21": "LABEL_21",
    "22": "LABEL_22",
    "23": "

In [81]:
tok_clf_model.config.id2label[0], tok_clf_model.config.id2label[1]

('B-genre', 'B-year')

In [82]:
epochs = 2

training_args = TrainingArguments(
    output_dir="./snips_tok_clf/results",
    num_train_epochs=epochs,
    per_device_train_batch_size=32,
    per_device_eval_batch_size=32,
    load_best_model_at_end=True,
        
    logging_steps=10,
    log_level='info',
    evaluation_strategy='epoch',
    save_strategy='epoch'
)

# Define the trainer:

trainer = Trainer(
    model=tok_clf_model,
    args=training_args,
    train_dataset=tok_clf_tokenized_snips['train'],
    eval_dataset=tok_clf_tokenized_snips['test'],
    data_collator=tok_data_collator
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).


In [83]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32


{'eval_loss': 4.404926776885986,
 'eval_runtime': 34.8636,
 'eval_samples_per_second': 75.064,
 'eval_steps_per_second': 2.352}

In [84]:
trainer.train()

***** Running training *****
  Num examples = 10467
  Num Epochs = 2
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 656


Epoch,Training Loss,Validation Loss
1,0.256700,0.178535
2,0.099100,0.134331


***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
Saving model checkpoint to ./snips_tok_clf/results/checkpoint-328
Configuration saved in ./snips_tok_clf/results/checkpoint-328/config.json
Model weights saved in ./snips_tok_clf/results/checkpoint-328/pytorch_model.bin
***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32
Saving model checkpoint to ./snips_tok_clf/results/checkpoint-656
Configuration saved in ./snips_tok_clf/results/checkpoint-656/config.json
Model weights saved in ./snips_tok_clf/results/checkpoint-656/pytorch_model.bin


Training completed. Do not forget to share your model on huggingface.co/models =)


Loading best model from ./snips_tok_clf/results/checkpoint-656 (score: 0.13433107733726501).


TrainOutput(global_step=656, training_loss=0.43208915912886947, metrics={'train_runtime': 1167.7169, 'train_samples_per_second': 17.927, 'train_steps_per_second': 0.562, 'total_flos': 115900561605024.0, 'train_loss': 0.43208915912886947, 'epoch': 2.0})

In [85]:
trainer.evaluate()

***** Running Evaluation *****
  Num examples = 2617
  Batch size = 32


{'eval_loss': 0.13433107733726501,
 'eval_runtime': 32.8603,
 'eval_samples_per_second': 79.64,
 'eval_steps_per_second': 2.495,
 'epoch': 2.0}

In [109]:
pipe = pipeline("token-classification", tok_clf_model, tokenizer=tokenizer)
pipe('Add Two Coins by Dispatch to my road trip playlist')

[{'entity': 'B-entity_name',
  'score': 0.7401369,
  'index': 2,
  'word': 'two',
  'start': 4,
  'end': 7},
 {'entity': 'I-entity_name',
  'score': 0.9019552,
  'index': 3,
  'word': 'coins',
  'start': 8,
  'end': 13},
 {'entity': 'B-artist',
  'score': 0.9180974,
  'index': 5,
  'word': 'dispatch',
  'start': 17,
  'end': 25},
 {'entity': 'B-playlist_owner',
  'score': 0.9877879,
  'index': 7,
  'word': 'my',
  'start': 29,
  'end': 31},
 {'entity': 'B-playlist',
  'score': 0.9911526,
  'index': 8,
  'word': 'road',
  'start': 32,
  'end': 36},
 {'entity': 'I-playlist',
  'score': 0.99032265,
  'index': 9,
  'word': 'trip',
  'start': 37,
  'end': 41}]

In [114]:
pipe = pipeline("token-classification", tok_clf_model, tokenizer=tokenizer)
pipe('Rate The Principles of Data Science 5 out of 5')

[{'entity': 'B-object_name',
  'score': 0.9802408,
  'index': 2,
  'word': 'the',
  'start': 5,
  'end': 8},
 {'entity': 'I-object_name',
  'score': 0.9885514,
  'index': 3,
  'word': 'principles',
  'start': 9,
  'end': 19},
 {'entity': 'I-object_name',
  'score': 0.99320906,
  'index': 4,
  'word': 'of',
  'start': 20,
  'end': 22},
 {'entity': 'I-object_name',
  'score': 0.99143934,
  'index': 5,
  'word': 'data',
  'start': 23,
  'end': 27},
 {'entity': 'I-object_name',
  'score': 0.99307764,
  'index': 6,
  'word': 'science',
  'start': 28,
  'end': 35},
 {'entity': 'B-rating_value',
  'score': 0.9917738,
  'index': 7,
  'word': '5',
  'start': 36,
  'end': 37},
 {'entity': 'B-best_rating',
  'score': 0.9863114,
  'index': 10,
  'word': '5',
  'start': 45,
  'end': 46}]

## 6.4 BERT for question/answering

In [43]:
from transformers import DistilBertTokenizerFast, DistilBertForQuestionAnswering, Trainer, TrainingArguments, AutoTokenizer, AutoModelForQuestionAnswering
from datasets import Dataset, load_dataset
import pandas as pd

import seaborn as sns
import matplotlib.pyplot as plt

In [44]:
qa_bert = AutoModelForQuestionAnswering.from_pretrained("distilroberta-base")

bert_tokenizer = AutoTokenizer.from_pretrained("distilroberta-base")

loading configuration file https://huggingface.co/distilroberta-base/resolve/main/config.json from cache at /Users/sinanozdemir/.cache/huggingface/transformers/42d6b7c87cbac84fcdf35aa69504a5ccfca878fcee2a1a9b9ff7a3d1297f9094.aa95727ac70adfa1aaf5c88bea30a4f5e50869c68e68bce96ef1ec41b5facf46
Model config RobertaConfig {
  "_name_or_path": "distilroberta-base",
  "architectures": [
    "RobertaForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "bos_token_id": 0,
  "classifier_dropout": null,
  "eos_token_id": 2,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-05,
  "max_position_embeddings": 514,
  "model_type": "roberta",
  "num_attention_heads": 12,
  "num_hidden_layers": 6,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.15.0",
  "type_vocab_size": 1,
  "use_cache": true,
  "vocab_size": 50265
}

loading weights file https://hugg

In [12]:
# # there are pre fine-tuned models that learned from the SQuAD dataset
# bert_tokenizer = BertTokenizerFast.from_pretrained(
#     'bert-large-uncased-whole-word-masking-finetuned-squad', return_token_type_ids=True
# )
# qa_bert = BertForQuestionAnswering.from_pretrained('bert-large-uncased-whole-word-masking-finetuned-squad')

In [45]:
# This code snippet parses a dataset from Huggingface to give the four parameters we need for Q/A:
#  question (str), context (str), start_positions (int), end_positions (int)

q, c, s, e, f_a = [], [], [], [], []

def get_sub_list_positions(context_encoded, answer_encoded):
    for idx in range(len(context_encoded) - len(answer_encoded) + 1):
        if context_encoded[idx : idx + len(answer_encoded)] == answer_encoded:
            return idx, idx + len(answer_encoded) - 1
    return None, None
    
for example in load_dataset('adversarial_qa', 'adversarialQA', split='train'):
    context_encoded = bert_tokenizer.encode(example['question'], example['context'])
    for answer, answer_start in zip(example['answers']['text'], example['answers']['answer_start']):
        answer_encoded = bert_tokenizer.encode(answer, add_special_tokens=False)  # ignore the CLS and SEP token
        start_pos, end_pos = get_sub_list_positions(context_encoded, answer_encoded)
        if start_pos:
            q.append(example['question'])
            c.append(example['context'])
            s.append(start_pos)
            e.append(end_pos)
            f_a.append(answer)
            break

qa_df = pd.DataFrame({
    'question': q, 'context': c, 'start_positions': s, 'end_positions': e, 'answer':  f_a
})

Reusing dataset adversarial_qa (/Users/sinanozdemir/.cache/huggingface/datasets/adversarial_qa/adversarialQA/1.0.0/92356be07b087c5c6a543138757828b8d61ca34de8a87807d40bbc0e6c68f04b)
Token indices sequence length is longer than the specified maximum sequence length for this model (548 > 512). Running this sequence through the model will result in indexing errors


In [46]:
qa_df.shape

(2266, 5)

In [47]:
# only grab 2,000 examples
qa_dataset = Dataset.from_pandas(qa_df.sample(2000, random_state=42))

# train test split method
qa_dataset = qa_dataset.train_test_split(test_size=0.2)

qa_dataset['train'][0]

{'question': "Why did Albert's name hold sympathetic value?",
 'context': 'His birthday (14 December 1895) was the 34th anniversary of the death of his great-grandfather, Prince Albert, the Prince Consort. Uncertain of how the Prince Consort\'s widow, Queen Victoria, would take the news of the birth, the Prince of Wales wrote to the Duke of York that the Queen had been "rather distressed". Two days later, he wrote again: "I really think it would gratify her if you yourself proposed the name Albert to her". Queen Victoria was mollified by the proposal to name the new baby Albert, and wrote to the Duchess of York: "I am all impatience to see the new one, born on such a sad day but rather more dear to me, especially as he will be called by that dear name which is a byword for all that is great and good". Consequently, he was baptised "Albert Frederick Arthur George" at St. Mary Magdalene\'s Church near Sandringham three months later.[a] As a great-grandson of Queen Victoria, he was known 

In [48]:
def preprocess(data):
    return bert_tokenizer(data['question'], data['context'], truncation=True, max_length=256)

qa_dataset = qa_dataset.map(preprocess, batched=True)

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/1 [00:00<?, ?ba/s]

In [49]:
qa_dataset = qa_dataset.remove_columns(
    ['question', 'context', 'answer', '__index_level_0__']
)

qa_dataset

DatasetDict({
    train: Dataset({
        features: ['start_positions', 'end_positions', 'input_ids', 'attention_mask'],
        num_rows: 1600
    })
    test: Dataset({
        features: ['start_positions', 'end_positions', 'input_ids', 'attention_mask'],
        num_rows: 400
    })
})

In [50]:
batch_size = 32
epochs = 4

training_args = TrainingArguments(
    output_dir='./qa/results',
    num_train_epochs=epochs,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    logging_dir='./qa/logs',
    save_strategy='epoch',
    logging_steps=50,
    evaluation_strategy='epoch',
    load_best_model_at_end=True
)

trainer = Trainer(
    model=qa_bert,
    args=training_args,
    train_dataset=qa_dataset['train'],
    eval_dataset=qa_dataset['test'],
    tokenizer=bert_tokenizer
)

# Get initial metrics
trainer.evaluate()

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
***** Running Evaluation *****
  Num examples = 400
  Batch size = 32


KeyboardInterrupt: 

In [ ]:
trainer.train()

In [ ]:
trainer.evaluate()

In [ ]:
trainer.save_model()

In [ ]:
pipe = pipeline("question-answering", './qa/results', tokenizer=bert_tokenizer)

pipe("Where is Sinan living these days?", "Sinan lives in California but Matt lives in Boston.")

In [ ]:
pipe("Where is Matt living these days?", "Sinan lives in California but Matt lives in Boston.")

In [ ]:
princeton = """In 1675, a Quaker missionary from England, encouraged by New Jersey proprietors John Lord 
              "Berkeley and Sir George Carteret, arrived to establish a settlement in this area near the 
              "Delaware River, which was inhabited by the Lenni-Lenape Indians. The Keith survey of 1685 
              "established the western boundary of Middlesex and Somerset Counties and later, the Township 
              "of Princeton. Today Keith's Line is recognized as Province Line Road. With the laying of the 
              "cornerstone for Nassau Hall in 1754, Princeton began its development as a location for 
              "quality education. Nassau Hall was named for William III, Prince of Orange-Nassau. This simple stone 
              "edifice was one of the largest public buildings in the colonies and became a model for many other 
              "structures in New Jersey and Pennsylvania."""

pipe("Why was Princeton founded?", princeton)

In [ ]:
qa_input = bert_tokenizer(
    "Where is Matt living these days?", "Sinan lives in California but Matt lives in Boston.",
    return_tensors='pt'
)
qa_input

In [ ]:
output = qa_bert(**qa_input)
output

In [ ]:
token_labels = bert_tokenizer.convert_ids_to_tokens(qa_input['input_ids'].squeeze())
token_labels[:5]

In [ ]:
sns.set(rc={"figure.figsize":(20, 5)}) 


# Create a barplot showing the start word score for all of the tokens.
ax = sns.barplot(x=[f'{i}_{t}' for i, t in enumerate(token_labels)], y=output.start_logits.squeeze().tolist(), ci=None)
# Turn the xlabels vertical.
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")
# Turn on the vertical grid to help align words to scores.
plt.title('Start Word Scores')

plt.show()

# Create a barplot showing the start word score for all of the tokens.
ax = sns.barplot(x=[f'{i}_{t}' for i, t in enumerate(token_labels)], y=output.end_logits.squeeze().tolist(), ci=None)
# Turn the xlabels vertical.
ax.set_xticklabels(ax.get_xticklabels(), rotation=90, ha="center")
# Turn on the vertical grid to help align words to scores.
plt.title('End Word Scores')

plt.show()